# LDA - RF Pipeline 

## Import necessary packages

In [1]:
import time
import optuna
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
# from bitermplus import BTM

from dataset.dataset import Dataset
from constants import CLEANED_DATASET_PATH

## Constants

In [2]:
RANDOM_SEED = 0

## Load the Hate Speech Filipino dataset from Hugging Face

In [3]:
dataset = Dataset(full_data_path=CLEANED_DATASET_PATH,
                  from_scratch=False,
                  split_sizes=[10000, 4232, 4232])
dataset.build()

X_train = dataset.get_features(split_type="train")
Y_train = dataset.get_labels(split_type="train")
X_val = dataset.get_features(split_type="val")
Y_val = dataset.get_labels(split_type="val")
X_test = dataset.get_features(split_type="test")
Y_test = dataset.get_labels(split_type="test")

Data loaded from dataset/cleaned_dataset.pkl


In [4]:
# Re-split the dataset into training, validation, and test sets
# X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)
# X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=RANDOM_SEED)

## Vectorize the texts to be able to perform LDA

In [5]:
# Define the CountVectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=2)
X_train_vector = vectorizer.fit_transform(X_train)
X_val_counts = vectorizer.transform(X_val)
X_test_counts = vectorizer.transform(X_test)

## Perform Latent Dirichlet Allocation on the training set

In [6]:
# N_TOPICS = 10
# print(f"Performing Latent Dirichlet Allocation for {N_TOPICS} topics")
# lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=RANDOM_SEED)
# X_train_topics = lda.fit_transform(X_train_vector)
# X_val_topics = lda.transform(X_val_counts)
# X_test_topics = lda.transform(X_test_counts)
# print(f"Done performing Latent Dirichlet Allocation for {N_TOPICS} topics")

## Perform Non-negative Matrix Factorization on the training set

In [7]:
N_TOPICS = 20
print(f"Performing Non-negative Matrix Factorization for {N_TOPICS} topics")
nmf = NMF(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = nmf.fit_transform(X_train_vector)
X_val_topics = nmf.transform(X_val_counts)
X_test_topics = nmf.transform(X_test_counts)
print(f"Done performing Non-negative Matrix Factorization for {N_TOPICS} topics")

Performing Non-negative Matrix Factorization for 20 topics
Done performing Non-negative Matrix Factorization for 20 topics


## Search for the best hyperparameters of the Random Forest model

In [8]:
# Define the objective function for Optuna
def create_objective(X_train, Y_train, X_test, Y_test):
    def objective(trial):
        n_estimators = trial.suggest_int("n_estimators", 100, 1000)
        rf = RandomForestClassifier(n_estimators=n_estimators, 
                                    random_state=RANDOM_SEED)
        rf.fit(X_train, Y_train)
        Y_pred = rf.predict(X_test)
        score = accuracy_score(Y_test, Y_pred)
        return score
    return objective

In [9]:
# Create a study object
# study = optuna.create_study(direction="maximize", study_name=f"LDA_RF_Pipeline")
study = optuna.create_study(direction="maximize", study_name=f"NMF_RF_Pipeline")


# Create the study objective
objective = create_objective(X_train_topics, Y_train, X_val_topics, Y_val)

# Execute an optimization
study.optimize(objective, n_trials=20, n_jobs=-1)

# Print the best trial results
print(f"Best Accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2024-04-30 23:29:44,005] A new study created in memory with name: LDA_RF_Pipeline
[I 2024-04-30 23:29:52,195] Trial 2 finished with value: 0.6810018903591682 and parameters: {'n_estimators': 250}. Best is trial 2 with value: 0.6810018903591682.
[I 2024-04-30 23:29:52,321] Trial 0 finished with value: 0.6814744801512287 and parameters: {'n_estimators': 255}. Best is trial 0 with value: 0.6814744801512287.
[I 2024-04-30 23:29:53,839] Trial 3 finished with value: 0.6819470699432892 and parameters: {'n_estimators': 291}. Best is trial 3 with value: 0.6819470699432892.
[I 2024-04-30 23:29:55,395] Trial 4 finished with value: 0.6814744801512287 and parameters: {'n_estimators': 335}. Best is trial 3 with value: 0.6819470699432892.
[I 2024-04-30 23:29:57,633] Trial 10 finished with value: 0.6810018903591682 and parameters: {'n_estimators': 402}. Best is trial 3 with value: 0.6819470699432892.
[I 2024-04-30 23:29:59,008] Trial 7 finished with value: 0.6805293005671077 and parameters: {'n_est

Best Accuracy: 0.68265595463138
Best hyperparameters: {'n_estimators': 511}


In [10]:
best_n_estimators = study.best_trial.params["n_estimators"]

rf = RandomForestClassifier(n_estimators=best_n_estimators, random_state=RANDOM_SEED)
rf.fit(X_train_topics, Y_train)
Y_pred = rf.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
# print(f"Best LDA-RF Model Accuracy: {score}")
print(f"Best NMF-RF Model Accuracy: {score}")

Best NMF-RF Model Accuracy: 0.6767485822306238
